# Notebook to estimate performance of a DRT system for many different parameter constellations
- estimations bases on previous simulation of the shortest distance to connect n random points (in any order) within unit circle
- formula when drawing n (number of passengers) out z random bus stops (with equal probability) 

In [209]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import datetime

# 3D
from mpl_toolkits.mplot3d import Axes3D 

# monitor memory space utility
import os, psutil # os not needed


In [210]:
# helper functions

def flattenList(l, degree:int=1):
    """
    Helper function to flat list degree times
    """
    if n_iters == 0: return(l)
    return [item for sublist in flattenList(l, degree-1) for item in sublist]
#

def listProduct(ar_list):
    """
    Helper Function to create all possible combination of multiple lists
    """
    if not ar_list:
        yield ()
    else:
        for a in ar_list[0]:
            for prod in listProduct(ar_list[1:]):
                yield (a,)+prod
#

def multi_indexer_assigner(d:dict, indexer_arr:list, value):
        if len(indexer_arr)<2:
            d[indexer_arr[0]]=value
            return
        multi_indexer_assigner(d[indexer_arr.pop(0)], indexer_arr, value)
#

def multi_indexer(d_arr:dict, indexer_arr:list):
    """
    iterates through indexes and return final element in d_arr
    """
    return(d_arr[indexer_arr[0]] if len(indexer_arr)<2 else multi_indexer_assigner(d[indexer_arr[0]], indexer_arr[1:], value))
#

def param_items_to_str(item_list:list):
    return ' '.join([str(key)+':'+str(vals[0] if type(vals)==list else vals) for (key,vals) in item_list])
#

def scale_increaser(n:float, scale_items:list):
    for scale_factor, scale_unit in scale_items:
        if n<scale_factor:
            break 
        n = n/scale_factor
    return (n, scale_unit)
#

def print_scenario_estimation(n_combinations, total_memory:int=psutil.Process().memory_info().rss):
    """
    prints a guess of computing resources needed to compute scenario
     
    Inputs:
        - n_combinations: number of scenario combinations
        - total_memory: in byte

    does not include fact that base params (e.g number of passengers) increase the scenarios possibilites by more than e.g. vehicle velocity
    """
    time_est = n_combinations*43.2/640000//1
    time_scale = [(60,'seconds'),(60,'minutes'),(24,'hours'),(7,'days'),(4.3,'weeks'),(12,'months'),(0,'years')]
    
    byte_est = 350*1e6/512000*n_combinations//1
    byte_scale = [(1e3, 'byte'),(1e3, 'kb'),(1e3, 'mb'),(1e3, 'gb'),(1e3, 'tb')]
    print("time_est, byte_est",time_est, byte_est)
    print( 'Number of scenario combinations:', n_combinations,
        '\nEstimated computing time:', scale_increaser(time_est, time_scale),
        '\nEstimated memory space usage:', scale_increaser(byte_est, byte_scale), ' equals' , byte_est/psutil.virtual_memory().total*1000//1/10, 'percent'
    )
#




In [211]:
# possible params:
n_stops_in_net = 50 # in h
n_pax = 12 # number of pax per tour OR average number of accepted request per hour OR average vehicle occupancy in h, pax leaves vehicle after travelling 10km
direct_trip_length = 8 # distance each passengers travels on average in kmh until leaving vehicle
stop_time_first_pax = 0.004165 # stop time when 1 passenger is boarding or alighting in h
stop_time_extra_pax = 0.001666 # stop time increase for each addtional passenger boarding or alighting in h
velocity = 30.0 # in km/h
n_vehicles = 1
vehicle_capacity = 8
vehicle_km_per_day = 160
maximum_tour_duration = 4.5# pausezeiten?
break_duration = 0.75 #

In [212]:
# Function to apply on scenario parameters


def vehicle_cost_calc(vehicle_capacity:int=vehicle_capacity, vehicle_km_per_day:float=vehicle_km_per_day)->tuple:
    """
    return tuple of 
        - total cost
        - dict of
            - vehicle fix cost per day in eur, 
            - vehicle km based cost in eur, 
            - personal cost per day in eur
            - personal cost share
    Inputs:
        - vehicle_capacity
        - km per day
    """
    fix_cost=(6+.042*vehicle_capacity*1.05**vehicle_capacity**.8) * 1000 / 365
    km_cost=(13+.15*vehicle_capacity*1.05**vehicle_capacity**.8 **.8) * 1000 * (vehicle_km_per_day / 75000)
    staff_cost=33.6 * 1000 / 365
    total_cost = fix_cost+km_cost+staff_cost
    # staff_cost_share should be: 0.48:26-38 seats, 0.39:39-55 seats, 0.38:26-38 seats # Tabelle 1: Kostenstrukturen spanischer Fernbusunternehmen in Abhängigkeit der Busgröße [MiFo13, S. 8, 12, 16] 
    return (total_cost, {'fix_cost':fix_cost, 'km_cost': km_cost, 'staff_cost':staff_cost, 'staff_cost_share':staff_cost/total_cost, 'marginal_cost_per_km':km_cost/vehicle_km_per_day})
#

staff_cost_share = vehicle_cost_calc()[1]['staff_cost_share']
#

def n_stops_on_path(n_pax:int=n_pax, n_stops_in_net:int=n_stops_in_net, stop_demand_fun=None) -> float:
    """
    returns expected number of unique stops rounded up to nearest int
    Inputs:
        - n_pax: number of individual passengers
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
    """
    # def ceil(f): return int(f//1+(f%1>0))
    # return ceil(n_stops_in_net*(1-((n_stops_in_net-1)/n_stops_in_net)**(2*n_pax)))
    return n_stops_in_net*(1-((n_stops_in_net-1)/n_stops_in_net)**(2*n_pax))
    # potentially let demand at each stop be drawn from distribution e.g. uniform(0-1)
#

##### TODO work on this functions. its decreasing too early.
def dist_total(n_stops_on_path:int=n_stops_on_path()) -> float:
    """
    return expected total travel distance
    Inputs:
        - n_stops_on_path: number of unique stops on path
    
    Function is an approximation to the resultion route length with respect of number of random points to connect within unit circle
    """
    return (1.2*(n_stops_on_path-1)/(n_stops_on_path**.5) + 0.1*n_stops_on_path - 0.0037*n_stops_on_path**1.7)*direct_trip_length
    #7*n_stops_on_path**0.2-0.05*n_stops_on_path+0.15*n_stops_on_path*0.99**n_stops_on_path-25
#

def stop_time_total(
        n_pax:int=n_pax, 
        n_stops_on_path:int=n_stops_on_path(), 
        stop_time_first_pax:float=stop_time_first_pax, 
        stop_time_extra_pax:float=stop_time_extra_pax
        ) -> float:
    """
    returns total stop time
    Inputs:
        - n_pax: number of individual passengers
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
    """
    return stop_time_first_pax*n_stops_on_path + stop_time_extra_pax*(2*n_pax-n_stops_on_path)
#

def share_driving_time(
        velocity:float=velocity,
        dist_total=dist_total(),
        stop_time_total=stop_time_total()
        ) -> float:
    """
    returns the share of time the vehicle is moving
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    # return dist_total/(dist_total+stop_time_total*velocity)
    return (dist_total/velocity)/(dist_total/velocity+stop_time_total)
#

def average_travel_speed(
        velocity:float=velocity,
        share_driving_time=share_driving_time()
        ) -> float:
    """
    returns the average travel speed of vehicle per operating hour
    Inputs:
        - velocity: vehicle velocity when travelling
    """
    # return dist_total/(dist_total/velocity+stop_time_total)
    return share_driving_time*velocity
#

def pax_served_per_hour(
        n_pax:int=n_pax, 
        velocity:float=velocity,
        dist_total=dist_total(),
        stop_time_total=stop_time_total()
        ) -> float:
    """
    returns the average travel speed of vehicle per operating hour
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    return n_pax/(dist_total/velocity+stop_time_total)
#

def pax_trip_time_average(
        dist_total=dist_total(),
        average_travel_speed=average_travel_speed()
        ) -> float:
    """
    returns the average travel speed of vehicle per operating hour
    Inputs:
        - velocity: vehicle velocity when travelling
        - dist_total: total travel distance travelled to serve all pax
        - stop_time_total: total time stopping for boarding and alighting pax
    """
    return dist_total/average_travel_speed
#
    

In [213]:
# label dicts for var and functions

# ['n_stops_on_path', 'dist_total', 'stop_time_total', 'share_driving_time', 'average_travel_speed', 'pax_served_per_hour']
kpi_to_label = {
    'n_stops_on_path': 'Expeted number of unique stops',
    'dist_total': 'Total length of shortest path connecting all points',
    'stop_time_total': 'Total bus time stop time',
    'share_driving_time': 'Share of driving time of total bus operating time',
    'average_travel_speed': 'Average speed of bus during operating time',
    'pax_served_per_hour': 'Passengers served per hour',
    'pax_trip_time_average': 'Average travel time per trip'
}

# ['n_pax', 'n_stops_in_net', 'direct_trip_length', 'stop_time_first_pax', 'stop_time_extra_pax', 'velocity', 'stop_demand_fun']
param_names_to_label = {
    'n_pax': 'Number of Passengers',
    'n_stops_in_net': 'Total number of stops within network',
    'direct_trip_length': 'Length of direct OD-path in km',
    'stop_time_first_pax': 'Stop time for single PAX enter/exit in mins',
    'stop_time_extra_pax': 'Add stop time for add PAX enter/exit in mins',
    'velocity': 'Free floating speed of vehicle',
    'stop_demand_fun': 'Function of demand distribution among stops',
}

key_to_label = dict(list(kpi_to_label.items())+list(param_names_to_label.items()))

In [214]:
def sci_params_generator(
        n_pax:list or int=[10], 
        n_stops_in_net:list or int=[100],
        direct_trip_length:list or float = [9.1],# distance each passengers travels on average in kmh until leaving vehicle
        stop_time_first_pax:list or float = [0.004165], # stop time when 1 passenger is boarding or alighting in h
        stop_time_extra_pax:list or float = [0.001666], # stop time increase for each addtional passenger boarding or alighting in h
        velocity:list or float = [30.0], # in km/h
        stop_demand_fun = [None],
)->dict:

        # ensure all params are provided within array
        param_name_to_sci_val = dict([(key, val if type(val)==list else [val]) for (key, val) in locals().items()])

        # estimation of computing times
        print_scenario_estimation(math.prod([len(x) for x in param_name_to_sci_val.values()]))

        return param_name_to_sci_val

In [215]:
def scenario_controller(
        run:list or bool=True,
        n_iterations: int=1,
        create_plots: bool=False,
        param_name_to_sci_val:dict=sci_params_generator(),
        kpi_funcs_ordered = [n_stops_on_path, dist_total, stop_time_total, share_driving_time, average_travel_speed, pax_served_per_hour, pax_trip_time_average]
    )->list:
    """
    calls different scenario aspects and stores different results
    Inputs:
        - run: bool or list. If False then only number of compinations for scenario is returned. If True all aspects and functions are run. Or vector strings with of function names that should be exececuted
        - n_iterations
        - create_plots
        - n_pax: ??number of pax per tour OR average number of accepted request per hour OR average vehicle occupancy in h
        - n_stops_in_net: number of stops where pax could potentially picked up / dropped off 
        - direct_trip_length: length of direct path between origin and demand of any pax
        - stop_time_first_pax: stop time when 1 passenger is boarding or alighting in h
        - stop_time_extra_pax: stop time increase for each addtional passenger boarding or alighting in h
        - velocity: vehicle velocity in km/h when driving

    Abbreivations: sci: scenarios, sce: scenario, param: modifiable scenario parameter, 

    kpi_name and kpis should be renamed to what they actually are: they compute key indicators for given cenario
    """
    # unpack parameter dictionary for easier later reference
    param_name_vals_sci, all_param_names = list(param_name_to_sci_val.items()), list(param_name_to_sci_val.keys())
    

    if not run: # early return when not running
        return

    # store list with names for later reference to avoid retrieving the name in nested loops
    kpi_names = [kpi.__name__ for kpi in kpi_funcs_ordered]
    # for each kpi determine recursively what its direct function inputs and what its influential factors are
    kpi_to_infl_factors, kpi_to_direct_func_inputs = {}, {}
    for kpi_name, kpi in zip(kpi_names, kpi_funcs_ordered):
        kpi_direct_func_inputs = list(kpi.__code__.co_varnames[:kpi.__code__.co_argcount])
        kpi_to_direct_func_inputs[kpi_name] = kpi_direct_func_inputs

        # flatten once
        kpi_influences_unsorted_duplicated = [item for sublist in 
                [[param_or_kpi_name] if param_or_kpi_name in all_param_names else kpi_to_infl_factors[param_or_kpi_name] for param_or_kpi_name in kpi_direct_func_inputs] 
            for item in sublist]
        kpi_to_infl_factors[kpi_name] = sorted(list(set(kpi_influences_unsorted_duplicated)), key=all_param_names.index)
    #

    kpi_params_arr = [(kpi_name, sorted(list(set(all_param_names) & set(kpi_to_infl_factors[kpi_name])), key=all_param_names.index)) for kpi_name in kpi_names]
    kpi_items_arr = [(kpi_name, [(param_name, param_name_to_sci_val[param_name]) for param_name in param_names]) for (kpi_name, param_names) in kpi_params_arr]

    # memory error is likely as the results for all the function are held in memory at the same time
    # holding results for function for which the outputs are needed for later functions is however necessary
    # a way to prevent this is to generate a functional form that directly return the results with respect to influence parameters    
    kpi_to_out = dict([(kpi_name, {
        # param only refers to influencial factor for kpi # maybe rename. 
        'param_names': [param_name for (param_name,param_vals) in params_items], 
        'param_items': [(param_name,param_vals) for (param_name,param_vals) in params_items], 
        'param_names_to_vals': dict([(param_name,param_vals) for (param_name,param_vals) in params_items]), 
        'params_dim':[len(param_vals) for (param_name,param_vals) in params_items], 
        'params_val_to_kpi_val': {}
        }) for (kpi_name, params_items) in kpi_items_arr])
    # res_arr is constructed from outside to inside res_arr[i] references param_name_vals_sci[flex_params_dims[0]][i]
    # res_arr is constructed from outside to inside res_arr[i][j] references param_name_vals_sci[flex_params_dims[0]][i] with param_name_vals_sci[flex_params_dims[1]][j]
    # res_arr is constructed from outside to inside res_arr[i][j][k] references param_name_vals_sci[flex_params_dims[0]][i] with param_name_vals_sci[flex_params_dims[1]][j] with param_name_vals_sci[flex_params_dims[1]][j][k] 
    
    
    # prepare all combinations to loop through # maybe there is a better way to do it.
    sci_params_flat = [[(param_name, param_val) for param_val in param_vals] for (param_name, param_vals) in param_name_vals_sci]
    # create all possible combs
    param_val_combs_sci = [dict(x) for x in listProduct(sci_params_flat)]

    
    # store interim results to quickly check whether functions needs to be applied or whether result is already computed. Allows for quick request of result
    infl_factors_vals_to_kpi_val = {}
    
    for sce in param_val_combs_sci:
        for kpi_name, kpi_fun in zip(kpi_names, kpi_funcs_ordered):
            # optain list of param_name, param_value pairs of given scenario that are influencial factor to current kpi
            sce_param_names_n_vals_infl_kpi = [(param_name, sce[param_name]) for param_name in kpi_to_out[kpi_name]['param_names']]
            # str used to store and retrieve interim result for a kpi for combination of influencial factor values
            param_str = param_items_to_str(sce_param_names_n_vals_infl_kpi)
            
            # if the current combination of values of influencial factor already used to compute current kpi, jump to next kpi
            if (kpi_name+'='+param_str) in infl_factors_vals_to_kpi_val.keys():
                continue
            
            # get input parameter values for kpi function
            # values are either simply taken from sce parameter or 
            # if they refere to a kpi that has already been computed, then the result needs to be looked up according to current sce params that influence the precomputed kpi  
            kpi_func_direct_inputs = [
                sce[param_or_kpi_name]
                    if param_or_kpi_name in all_param_names else 
                infl_factors_vals_to_kpi_val[
                    (param_or_kpi_name+'='+param_items_to_str([(param_name, sce[param_name]) for param_name in kpi_to_out[param_or_kpi_name]['param_names']]))
                ] 
                for param_or_kpi_name in kpi_to_direct_func_inputs[kpi_name]
            ]
            # unpack input parameters to execute function and optain kpi value
            kpi_val = kpi_fun(*kpi_func_direct_inputs)
            # store as intermediate result
            infl_factors_vals_to_kpi_val[(kpi_name+'='+param_str)] = kpi_val
            # store in final result dictonary
            kpi_to_out[kpi_name]['params_val_to_kpi_val'][param_str] = kpi_val
        #
    #
    
    # return dictionary with results
    return {
        'kpi_to_out': kpi_to_out, 
        # 'param_name_vals_sci': param_name_vals_sci, 
        'param_name_to_sci_val': param_name_to_sci_val,
        # 'param_val_combs_sci': param_val_combs_sci,
        }
#


time_est, byte_est 0.0 683.0
Number of scenario combinations: 1 
Estimated computing time: (0.0, 'seconds') 
Estimated memory space usage: (683.0, 'byte')  equals 0.0 percent


In [216]:
param_name_to_sci_val = sci_params_generator(
    n_pax=[int(x) for x in np.linspace(1,80,80)], #(1,80,80)
    n_stops_in_net=[int(x) for x in np.linspace(20, 400, 20)],#(10, 200, 20)
    velocity=[x for x in np.linspace(24, 50, 1)], #(24, 50, 13)
    direct_trip_length=[x for x in np.linspace(9.1, 9.1, 1)], #(6, 13.5, 6)
    stop_time_first_pax=[x for x in np.linspace(20/60/60, 60/60/60, 10)],#(0.25, 1, 20)
    stop_time_extra_pax= [x for x in np.linspace(2/60/60, 10/60/60, 4)]#(2/60, 10/60, 9)
    )

sci_kpis = scenario_controller(
    run=True, 
    param_name_to_sci_val=param_name_to_sci_val
)

# filename
path, filename, ext = './scenarios/', 'sc_04_29', '.npy'

time_est, byte_est 4.0 43750000.0
Number of scenario combinations: 64000 
Estimated computing time: (4.0, 'seconds') 
Estimated memory space usage: (43.75, 'mb')  equals 0.5 percent


In [217]:
# saving does not work

# # Save scenarios
# print('saving scenarios to:',path+filename+ext)
# np.save(path+filename+ext, sci_kpis)

# # load scenarios
# sci_kpis = np.load(path+filename+ext, allow_pickle=True)

In [218]:
def dim_to_varname_generator(
        X='n_pax',
        Y=True,
        Z=None,
        L1='n_stops_in_net',
        L2=None,
        C=None,
        M=None,
    ):
        """
        True or 1 output should be plotted here
        None not used
        - varnames varnames ordered for plotting: [X,Y,Z,L1,L2,M,C] where
                - X: x-axis
                - Y: y-axis
                - Z: z-axis
                - L1: variable to create lines for each
                - L2: variable to create all line combinations with L1
                - M: create a subplot for each value of that variable
                - C: color variable
                length of list must be fixed to 7. None value has to be provided if field should be left out specifc field should be empty, it should state None
        """
        # what if same var on x and y? x and l1? X and C would not be a problem though.
        # To-Do: clean varnames dict
        # output cannot be on L1 or L2
        local_dict = locals()
        return local_dict

In [219]:
def rearrange_kpi_vals_to_dims(
        kpi_sci=sci_kpis['kpi_to_out'][list(sci_kpis['kpi_to_out'].keys())[0]], 
        dim_to_varname=dim_to_varname_generator(),
        param_name_to_vals_restr = None,
    ):
    """
    function to extract data corresponding to the specified varnames
    should return data dict
        - scen_dict_restr: (potentially restricted) scenario_dict with param values
    """
    # kpi val dictionary
    params_val_to_kpi_val = kpi_sci['params_val_to_kpi_val']
    # if not specified otherwise in param_name_to_vals_restr full parameter space is used
    param_names_to_vals = (param_name_to_vals_restr or kpi_sci['param_names_to_vals']).copy()
    # print("param_names_to_vals",param_names_to_vals)
    # unpack for easier reference
    param_names = param_names_to_vals.keys()
    
    # handle when multiple values for an input parameter are give but they are not assigned to any axis (X,Y,Z,L1,L2,M)
    passive_param_names = [param_name for param_name in param_names if not param_name in [param_name for (dim_name, param_name) in dim_to_varname.items() if not dim_name == 'C']]
    for (param_name, param_vals) in [(param_name, param_names_to_vals[param_name]) for param_name in passive_param_names if len(param_names_to_vals[param_name])>1]:
        print('WARNING: '+ param_name +' is misspecified as', len(param_vals), 'param values are but only first will be used:', param_vals[:1])
        # enforces passive parameters to be fixed
        param_names_to_vals[param_name] = param_vals[:1]
    #

    # selects only those parameters that are relevant for plotting, i.e. that have a dimension specified to them
    dim_to_active_param_name_vals = dict([(dim_name, (param_name, param_names_to_vals[param_name])) for (dim_name, param_name) in dim_to_varname.items() if param_name in param_names])    

    # check whether the order of dim_to_active_param_name_vals.items() is optimal or whether it should be sorted differently
    param_items_active = [item for (dim_key, item) in dim_to_active_param_name_vals.items()]
    param_items_active_long = [[(key, val) for val in vals] for (key, vals) in param_items_active]
    param_items_active_indexes_long = [[(key, i) for i in range(len(vals))] for (key, vals) in param_items_active]
    
    passive_param_to_val = dict([(param_name, param_names_to_vals[param_name]) for param_name in passive_param_names])
    # needs to contain all active combinations and fixed combinations
    # assume that only params specified for plot have variance. the other are list of length 1 and we select only entry

    # create all possible combinations
    param_items_active_combinations = [dict(x) for x in listProduct(param_items_active_long)]
    param_items_active_combinations_indexes = [[i for (key,i) in arr] for arr in listProduct(param_items_active_indexes_long)]
    
    # create empty frame for kpi_arr
    kpi_arr = np.zeros([len(vals) for (key, vals) in param_items_active])
    # print("kpi_arr",kpi_arr)
    for active_params_to_val_sce, scen_indexer in zip(param_items_active_combinations, param_items_active_combinations_indexes):
        # add params that are not active to have full lookup key
        params_val_sce = [(param_name, active_params_to_val_sce[param_name] if param_name in active_params_to_val_sce.keys() else passive_param_to_val[param_name]) for param_name in param_names]
        # lookup_key
        fetch_str = param_items_to_str(params_val_sce)
        # select data from dict
        fetched_res = params_val_to_kpi_val[fetch_str]
        # assign to kpi_arr array
        multi_indexer_assigner(kpi_arr, scen_indexer, fetched_res)
    #


    dim_to_vals = dict([(dim_key, np.asarray(param_vals)) for (dim_key, (param_name, param_vals)) in dim_to_active_param_name_vals.items()])
    dim_to_kpi = dict([(dim_key, kpi_arr) for (dim_key, param_name) in dim_to_varname.items() if param_name == True])
    dim_to_vals.update(dim_to_kpi)
    print('dim_to_vals',[(dim_name, len(dim_vals)) for (dim_name, dim_vals) in dim_to_vals.items()])
    # what other params to return?
    return {
        'dim_to_vals':dim_to_vals, 
        'passive_param_to_val':passive_param_to_val, 
        }
arranged_kpi_dict = rearrange_kpi_vals_to_dims()

dim_to_vals [('X', 80), ('L1', 20), ('Y', 80)]


In [239]:
def plotMultiDimensional(
        kpi_name = list(sci_kpis['kpi_to_out'].keys())[0],
        dim_to_varname:dict=dim_to_varname_generator(),
        plottitle:str=None, 
        contourlines:bool=False,
        expansionpath:bool=False,
        add_lines:list=[], 
        save:bool or str=False,
        sci_kpis=sci_kpis,
        ):
    """
    creates multidensional plot
    To-DO
        add possibility to fix one variable at specific value

    Inputs:
        - data: x dimensional np array
        
    """
    
    kpi_sci = sci_kpis['kpi_to_out'][kpi_name]
    param_names = kpi_sci['param_names']

    print('dim_to_varname',dim_to_varname)
    # what if same var on x and y? x and l1?
    arranged_kpi_dict = rearrange_kpi_vals_to_dims(
        kpi_sci=kpi_sci, 
        dim_to_varname=dim_to_varname, 
        param_name_to_vals_restr=None
        )
    dim_to_vals = arranged_kpi_dict['dim_to_vals']
    # print('dim_to_vals',[(dim_name, len(dim_vals)) for (dim_name, dim_vals) in dim_to_vals.items()])

    
    def arr_extr(dim_name:str, i_arr=[None,None,None], dim_to_vals:dict=dim_to_vals, dim_to_varname:dict=dim_to_varname):
        """
        s
        """
        # if empty return None
        if dim_to_varname[dim_name] == None:
                return None
        # if param simply return param values
        if dim_to_varname[dim_name] in param_names:
            return dim_to_vals[dim_name]
        # else nest more deeply to extract correct kpi values
        # wrap with list statement to not modify the original list
        d_arr = list(dim_to_vals[dim_name])
        print('d_arr',np.asarray(d_arr).shape)
        if i_arr[0]!=None:
                d_arr = [[b[i_arr[0]] for b in a] for a in dim_to_vals[dim_name]] # can be done with np functions..
        if i_arr[1]!=None:
                d_arr = [x[i_arr[1]] for x in dim_to_vals[dim_name]]
        if i_arr[2]!=None:
                d_arr = [x[i_arr[2]] for x in dim_to_vals[dim_name]]

        return np.asarray(d_arr)
    #

    
    # to-do check which option is the best
    plot_will_be_3D = not None in [dim_to_varname['X'],dim_to_varname['Y'],dim_to_varname['Z']]        
    
    C, minn, maxx, fcolors, c_dim_name = None, None, None, None, None
    # C = (dim_to_varname['C'] and dim_to_vals['C']) or (dim_to_varname['L1'] and dim_to_vals['L1']) or (dim_to_varname['L1'] and dim_to_vals['L1']) or None
    if dim_to_varname['C'] != None:
         C, c_dim_name = dim_to_vals['C'], 'C'
    elif dim_to_varname['L1'] != None:
         C, c_dim_name = dim_to_vals['L1'], 'L1'
    elif dim_to_varname['L2'] != None: 
         C, c_dim_name = dim_to_vals['L2'], 'L2'

    if c_dim_name:
        minn, maxx = C.min(), C.max() 
        norm = matplotlib.colors.Normalize(minn, maxx) 
        m = plt.cm.ScalarMappable(norm=norm, cmap='jet') 
        m.set_array([]) 
        fcolors = m.to_rgba(C)

    # set up 
    i_arr = [None, None, None]
    M_loop = list(enumerate(dim_to_vals['M'])) if dim_to_varname['M'] != None else [(None, None)]
    L1_loop =  list(enumerate(dim_to_vals['L1'])) if dim_to_varname['L1'] != None else [(None, None)]
    L2_loop =  list(enumerate(dim_to_vals['L2'])) if dim_to_varname['L2'] != None else [(None, None)]
    len_M = len(list(M_loop))
    nrows_M, ncols_M = math.ceil(len_M**.5), math.floor(len_M**.5)
    print("len_M",len_M, dim_to_varname['M'], M_loop )

    # create figure
    fig = plt.figure(figsize=(nrows_M*10,ncols_M*10))

    for (m_i, m_val) in M_loop:
        i_arr[0] = m_i
        label_str = m_i != None and str(m_val)
        _i = m_i != None and (m_i+1)

        ax = fig.add_subplot(nrows_M, ncols_M, _i or 1, projection='3d') if plot_will_be_3D else fig.add_subplot(nrows_M, ncols_M, _i)

        if add_lines:
             for _i, (line_y, line_m) in enumerate(add_lines):
                line_y + line_m
                xs=arr_extr('X',i_arr)
                ys=arr_extr('Y',i_arr)
                ax.plot(xs, [val_at_x if min(0,ys.min())<=val_at_x<=ys.max() else None for val_at_x in [(line_y+x*line_m) for x in xs]], '-.', color='black')
            #
        #
        for l1_i, l1_val in L1_loop:
            i_arr[1] = l1_i
            label_str = l1_i != None and str(l1_val)
            for l2_i, l2_val in L2_loop:
                i_arr[2] = l2_i
                if l2_i != None:
                    label_str = label_str + ", "+ str(l2_val)

                # if less than two parameters are plotted on the axis, simple linechart work
                if plot_will_be_3D:
                    # print('PLOT 3D')
                    x,y,Z = arr_extr('X',i_arr), arr_extr('Y',i_arr), arr_extr('Z',i_arr)
                    # convert to 2d matrices 
                    X, Y = np.meshgrid(y,x)
                    if X.shape[0]>X.shape[1]:
                        X = X.transpose()
                    if Y.shape[0]>Y.shape[1]:
                        Y = Y.transpose()
                    if Z.shape[0]>Z.shape[1]:
                        Z = Z.transpose()
                    # print(X.shape, Y.shape, Z.shape)
                    # create colormap according to x-value (can use any n_dimxn_dim array) 
                    if c_dim_name:
                        C = arr_extr(c_dim_name,i_arr) if 0 else np.zeros(X.shape)
                        if C.shape[0]>C.shape[1]:
                            C = C.transpose()
                        fcolors = m.to_rgba(C)

                    ax.plot_surface(
                             X,Y,Z, 
                            rstride=1, cstride=1,  
                            **({} if type(fcolors) not in [list, np.ndarray] and fcolors == None else {'facecolors':fcolors, 'vmin':minn, 'vmax':maxx}), 
                            shade=False, alpha=0.5
                        )
                elif len(list(set([dim_to_varname['X'], dim_to_varname['Y']])&set(param_names)))<2:
                    ax.plot(
                        arr_extr('X',i_arr), 
                        arr_extr('Y',i_arr),
                        label=label_str, 
                        color=fcolors if type(fcolors) not in [list, np.ndarray] else fcolors[{'L1':l1_i,'L2':l2_i}[c_dim_name]],# improve...
                        alpha=1.,
                        )
                    # marker='+', markersize=10, markerfacecolor="tab:blue", markeredgecolor="tab:blue"
                else: # otherwise heatmap chart is needed
                    C = arr_extr('C',i_arr)
                    # make sure dimension are ordered correctly
                    if (C.shape[0]>C.shape[1]) == (dim_to_vals['X'].shape[0]>dim_to_vals['Y'][0]):
                        C = C.transpose()

                    ax.contourf(arr_extr('X',i_arr), arr_extr('Y',i_arr), C, levels=20, cmap='viridis') # ax.imshow(...)
                    #
                #
            #
        #
        if label_str or c_dim_name:
            ax.legend(title="title")#fontsize='small'
            #
        #
        # add text: labels, titles, legend, etc
        if dim_to_varname['X']: 
            ax.set_xlabel((dim_to_varname['X'] in param_names and param_names_to_label[dim_to_varname['X']] or kpi_to_label[kpi_name]))
            ax.set_xlim(right=max(dim_to_vals['X']))
        if dim_to_varname['Y']: 
            ax.set_ylabel((dim_to_varname['Y'] in param_names and param_names_to_label[dim_to_varname['Y']] or kpi_to_label[kpi_name])) 
            # ax.set_ylim(bottom=min(dim_to_vals['Y']))
        if dim_to_varname['Z']: 
            ax.set_zlabel((dim_to_varname['Z'] in param_names and param_names_to_label[dim_to_varname['Z']] or kpi_to_label[kpi_name])) 
            # ax.set_ylim(bottom=min(dim_to_vals['Y']))
        #
        passive_param_str = "\n".join([param_names_to_label[param_name]+': '+str(param_vals[0]) for (param_name, param_vals) in arranged_kpi_dict['passive_param_to_val'].items()])
        ax.set_title(plottitle or kpi_to_label[kpi_name]+'\n'+passive_param_str)
        # ax.legend(handles=[line1, line2])
    #
        
    if save: print('save', save)
    return #fig
# plotMultiDimensional()


In [240]:
# ['n_pax', 'n_stops_in_net', 'direct_trip_length', 'stop_time_first_pax', 'stop_time_extra_pax', 'velocity', 'stop_demand_fun']


# plotMultiDimensional('n_stops_on_path',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ), add_lines=[(0,2)])
# plotMultiDimensional('dist_total',dim_to_varname=dim_to_varname_generator( X='n_pax', Y='n_stops_in_net', Z=None, L1=None, L2=None, C=True, M=None ))
# plotMultiDimensional('dist_total',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ))
# plotMultiDimensional('stop_time_total',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ), add_lines=[(0,2*0.00555)])
# plotMultiDimensional('share_driving_time',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1=None, L2=None, C=None, M='stop_time_first_pax' ))
plotMultiDimensional('share_driving_time',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M='stop_time_first_pax' ))
# plotMultiDimensional('share_driving_time',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ))
# plotMultiDimensional('average_travel_speed',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ))
# plotMultiDimensional('pax_served_per_hour',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M='stop_time_first_pax' ))
# plotMultiDimensional('pax_trip_time_average',dim_to_varname=dim_to_varname_generator( X='n_pax', Y=True, Z=None, L1='n_stops_in_net', L2=None, C=None, M=None ))

# plotMultiDimensional('share_driving_time',dim_to_varname=dim_to_varname_generator( X='n_pax', Y='n_stops_in_net', Z=True, L1=None, L2=None, C=None, M=None ))
# plotMultiDimensional('n_stops_on_path',dim_to_varname=dim_to_varname_generator( X='n_pax', Y='n_stops_in_net', Z=True, L1=None, L2=None, C=None, M=None ))
# plotMultiDimensional('share_driving_time',dim_to_varname=dim_to_varname_generator( X='n_pax', Y='n_stops_in_net', Z=True, L1=None, L2=None, C=True, M=None ))

dim_to_varname {'X': 'n_pax', 'Y': True, 'Z': None, 'L1': 'n_stops_in_net', 'L2': None, 'C': None, 'M': 'stop_time_first_pax'}
dim_to_vals [('X', 80), ('L1', 20), ('M', 10), ('Y', 80)]
len_M 10 stop_time_first_pax [(0, 0.005555555555555555), (1, 0.0067901234567901226), (2, 0.00802469135802469), (3, 0.009259259259259259), (4, 0.010493827160493827), (5, 0.011728395061728394), (6, 0.012962962962962963), (7, 0.01419753086419753), (8, 0.015432098765432098), (9, 0.016666666666666666)]
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_arr (80, 20, 10)
d_a

In [ ]:
# Plots: 
# Add 21*demand proportion line to n_pax + n_stops_in_net . Add x for cottbus and berlin and line
# Or should some kind of density measure be applied?

# Plot types
# Better have outcome variable on axis instead of color?=more clearly visible
# =if continous: add lines to color gradient s.t. you have contour lines or add markers along lines
# You could try to make it continous for each plot: add in between points for each dimension based on some average function. Best=get continous function to describe data.

# 2D
# 2D+color
# 2D multiline(1var, color gradient)
# 2D multiline(2vars,color gradient ^ alpha or line type)
# 3D+color
# 3D+color × multiple figures
# 3D+color+mutliple surface (maybe with arrows helping to understand the direction)
# 3D+color+mutliple surface (maybe with arrows helping to understand the direction at edge of picture)
# Add expansion path that hold a metric fixed
# n_stops_on_path
# 2D + color: n_pax + n_stops_in_net
# total_dist
# 2D + color: n_pax + n_stops_in_net
# 2D: n_stops_on_path = total_dist
# total_stop_time:
# 2D: n_stops_on_path
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# total_stop_time / pax
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# average_operating_speed
# 2D + color: n_pax + n_stops_in_net
# 4D + color: n_pax + time_first_pax + time_extra_pax (given n_stops_in_net)
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)
# pax_served_per_operating_hour
# 2D + color: n_pax + n_stops_in_net
# 3D + color: n_pax + n_stops_in_net
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)
# Optain combinations leading to 12 (cottbus), 24, 36 =cost efficieny, 48, 60 =lower cost
# def pax_trip_time_average() result from above only not normalized to 1 hour? *(n_pax -1)/(n_pax)
# If order matters path has to be traversed 1-2 times without 1 connection
# 2D + color: n_pax + n_stops_in_net
# 5D + color: n_pax + time_first_pax + time_extra_pax + velocity (given n_stops_in_net)